# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Using $p_h = h$ and combining the three constraints (2), (3) and (4) in order to get one constraint: 
$$m = rh + \tau^g\tilde{p_h} + \tau^p max[\tilde{p_h} - \bar{p} , 0]$$


Imports and set magics:

In [1]:
import numpy as np

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2


# local modules
import inauguralproject

# Question 1

BRIEFLY EXPLAIN HOW YOU SOLVE THE MODEL.

In [6]:
# Computing a dictionary of the parameter values
parameter_values = {'phi': 0.3, 'epsilon': 0.5, 'r': 0.03, 'tau_g': 0.012, 'tau_p': 0.004, 'p_bar': 3}


# Defining utility function 
def u_func(c, h, phi):
    """
    A function that computes the utillity of the household
        c: (float) Amount of consumption 
        h: (float) Amount of housing
        phi : (float) A parameter that determines the weights of h and c in the utility function
        
    Returns a float
    """
    return c**(1-phi)*h**phi







# Question 2

ADD ANSWER.

In [2]:
# anni ændrer i projektet

# Question 3

ADD ANSWER.

In [4]:
# code

# Question 4

ADD ANSWER.

In [5]:
# code

# Question 5

ADD ANSWER.

In [6]:
# code

# Conclusion

ADD CONCISE CONLUSION.